# 파이어베이스 (파이어스토어) 연결

https://firebase.google.com/docs/firestore?authuser=0


In [14]:
# cloud firestore 인스턴스 초기화(자체 서버에서 초기화)

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use the application default credentials - GCP에서 초기화
cred = credentials.Certificate('../firebase-apikey.json')
firebase_admin.initialize_app(cred)
db = firestore.client()


# 사용법


## 1. 참조
> users_ref = db.collection(u'users')

> docs_ref = db.collection(u'users').document(u'doc')

참조는 컬랙션 - 다큐먼트 - 컬랙션 - 다큐먼트 의 순서로만 가능하다

## 2. 데이터 쓰기

데이터 쓰기는 set() 메소드를 사용한다

> data = {
    u'name' : u'Los Angeles',
    u'state' : u'CA'
}

> db.collection(u'cities').document(u'LA').set(data)

유의미한 문서ID를 두고싶지 않을 때에는 add() 메소드를 사용한다. add() = .doc().set() <br>
*기존의 데이터를 그대로 두고 추가하고 싶다면 merge옵션을 True로 설정한다!!*

## 3. 데이터 업데이트 - 전체를 덮어쓰지 않고 일부 필드를 업데이트

업데이트는 update() 메소드를 사용한다<br>
배열 요소 업데이트는 arrayUnion()과 arrayRemove()를 사용한다.
> city_ref.update({u'regions' : firestore.ArrayUnion(\[u'greater_virginia'\])})

## 4. 데이터 가져오기

데이터 가져오기는 get() 메소드를 사용한다<br>
where() 메소드를 통해 컬랙션에서 특정 문서들만 뽑을 수 있다
> docs = db.collection(u'cities').where(u'capital', u'==', True).stream()

# 예시코드 (실행시 주의)

In [2]:
# 예시코드 - 콜랙션 내 모든 문서 불러오기

users_ref = db.collection(u'youtuber_keywords')
docs = users_ref.stream()

for doc in docs:
    print(u'{} => {}'.format(doc.id,doc.to_dict()))

youtube_keywords => {'youtube_title': '기리TV', 'keywords': ['플스5'], 'namuwiki_title': '기리TV'}


In [14]:
# 예시코드 - json 파일 읽고 db에 넣기

import json

with open('./crawling_result_text/youtuber_keywords.json', 'rt', encoding='utf-8') as f:
    json_data = json.load(f)

# print(json.dumps(json_data,ensure_ascii=False))


doc_ref = db.collection(u'youtuber').document(u'youtube_keywords')

for youtuber in json_data['youtubers']:
    doc_ref.set({
        u'youtube_title':youtuber['youtube_title'],
        u'namuwiki_title':youtuber['namuwiki_title'],
        u'keywords':youtuber['keywords']
    })


# 실제 코드(실행시 주의)

In [2]:
from googleapiclient.discovery import build
import os
import pandas as pd
import json


apiKey_dirt = 'APIkey.json'

#api 키 로드
with open( apiKey_dirt) as json_file:
    json_data = json.load(json_file)
    youtubeApiKey = json_data["APIkey"]

    
#채널 목록 로드
channel_list = pd.read_csv('./data_with_vling/youtuber_list.csv', header=0, sep=',')
channel_list.head(3)

,Unnamed: 0,name,youtube_code,namu_code,namu_url,txt_name
0,0,박닝겐,/channel/UCtG4WVb1P38zdKG8HXP4scQ,박닝겐,https://namu.wiki/w/박닝겐,박닝겐.txt
1,1,Xbox,/channel/UCjBp_7RuDBUYbd1LegWEJ8g,Xbox,https://namu.wiki/w/Xbox,Xbox.txt
2,2,Lime Tube[라임튜브],/channel/UCxutPqfWSdYqaK8_tlc5PBA,라임튜브,https://namu.wiki/w/라임튜브,라임튜브.txt


In [21]:
# 채널 정보 저장하기(신버전)

import re

youtube = build('youtube','v3',developerKey=youtubeApiKey)

error_youtuber = []

for i, channel_url in enumerate(channel_list['youtube_code']):
    if i < 603:
        continue
    #db 구조
    channel_data = {
        'youtube_title' : '',
        'namuwiki_title' : '',
        'channel_id' : '',
        'channel_desc' : '',
        'category' : [],
        'keywords' : '',
        'published_date' : '',
        'upload_playlist_id' : '',
        'channel_viewcount' : 0,
        'video_count' : 0,
        'subscriber_count' : 0,
        'trailer_video_id' : '',
        'defaultLanguage' : '',
        'country' : ''
    }

    #채널 id 저장
    channel_data['channel_id'] = re.sub(pattern='/channel/', repl='',string=channel_url)
    print('channel id : ' +channel_data['channel_id'] + ' clear')
    
    #나무위키 문서명 저장
    try:
        channel_data['namuwiki_title'] = channel_list['namu_code'][i]
        print('namuwiki title : ' + channel_data['namuwiki_title']+' clear')
    except:
        print('no namuwiki docs')

    #youtube api channel 메소드 연결
    channels_api = youtube.channels().list(part='id, snippet, brandingSettings, contentDetails,  statistics, topicDetails', id=channel_data['channel_id']).execute()

    #유튜브 채널명 저장
    try:
        channel_data['youtube_title'] = channels_api['items'][0]['brandingSettings']['channel']['title']
        print('youtube title : ' + channel_data['youtube_title']+' clear')
    except:
        print('please check this youtuber#################################')
        error_youtuber.append(channel_list['name'][i])
        continue

    #유튜버 소개글 저장
    try:
        channel_data['channel_desc'] = channels_api['items'][0]['brandingSettings']['channel']['description']
        print('channel description : ' +' clear')
    except:
        print('no description for this youtuber')

    # 채널 키워드 저장
    try:
        # keyword_list = channels_api['items'][0]['brandingSettings']['channel']['keywords'].split()
        # for keyword in keyword_list:
        #     channel_data['keywords'].append(keyword)
        # print("keywords : {}".format(channel_data['keywords']) + ' clear')
        channel_data['keywords'] = channels_api['items'][0]['brandingSettings']['channel']['keywords']
        print('keywords : ' + channel_data['keywords'] + ' okay')
    except :
        print("no keywords for this youtuber")

    # 카테고리 저장
    try:
        category_list =  channels_api['items'][0]['topicDetails']['topicCategories']
        for category in category_list:
            clean_category = re.sub(pattern='https://en.wikipedia.org/wiki/', repl='', string=category)
            channel_data['category'].append(clean_category)
        print('category : {}'.format(channel_data['category']) + ' clear')
    except:
        print('no category')

    #채널 시작일 저장
    published_date = channels_api['items'][0]['snippet']['publishedAt']
    channel_data['published_date'] = published_date
    print('published date : '+ channel_data['published_date']+' clear')

    #업로드 영상 재생목록 id 저장
    channel_data['upload_playlist_id'] = channels_api['items'][0]['contentDetails']['relatedPlaylists']['uploads']
    print('upload playlist id : ' + channel_data['upload_playlist_id'] + ' clear')

    #채널 조회수 저장
    channel_data['channel_viewcount'] = channels_api['items'][0]['statistics']['viewCount']
    print('viewcount : ' + channel_data['channel_viewcount']+ ' clear')

    #구독자수 저장
    try:
        channel_data['subscriber_count'] = channels_api['items'][0]['statistics']['subscriberCount']
        print('subscriber count : ' + channel_data['subscriber_count']+ ' clear')
    except:
        print('secret subscriber count')

    #비디오수 저장
    channel_data['video_count'] = channels_api['items'][0]['statistics']['videoCount']
    print('video count : ' + channel_data['video_count']+ ' clear')

    #트레일러 비디오 id 저장
    try:
        channel_data['trailer_video_id'] = channels_api['items'][0]['brandingSettings']['channel']['unsubscribedTrailer']
        print('trailer video id : ' + channel_data['trailer_video_id']+ ' clear')
    except:
        print('no trailer video')

    #기본 언어 저장
    try:
        channel_data['defaultLanguage'] = channels_api['items'][0]['brandingSettings']['channel']['defaultLanguage']
        print('default language : ' + channel_data['defaultLanguage']+ ' clear')
    except:
        print('maybe english')

    #국가 저장
    try:
        channel_data['country'] = channels_api['items'][0]['brandingSettings']['channel']['country']
        print('country : ' + channel_data['country']+ ' clear')
    except:
        print('maybe usa')

    print('@@@@@@@@@@@@@@@@@@@@' + channel_data['youtube_title'] + ' data crawling ok!')


    # firestore 저장
    doc_title = channel_data['youtube_title']
    doc_title = re.sub(pattern='/', repl=' ', string=doc_title)
    youtuber_ref = db.collection(u'Youtuber').document(doc_title)
    youtuber_ref.set(channel_data, merge=True)
    print('@@@@@@@@@@@@@@@@@@@@' +doc_title + ' firestore ok')
    print(error_youtuber)




    

channel id : UCCWcDTQwUzYxxQNxP17F-Rw clear
namuwiki title : 고래쉬 clear
youtube title : 고래쉬 clear
channel description :  clear
no keywords for this youtuber
category : ['Video_game_culture', 'Role-playing_video_game', 'Action_game'] clear
published date : 2013-03-30T13:00:04Z clear
upload playlist id : UUCWcDTQwUzYxxQNxP17F-Rw clear
viewcount : 45783873 clear
subscriber count : 94700 clear
video count : 535 clear
trailer video id : sGVUa-UaeOI clear
maybe english
maybe usa
@@@@@@@@@@@@@@@@@@@@고래쉬 data crawling ok!
@@@@@@@@@@@@@@@@@@@@고래쉬 firestore ok
[]
channel id : UCNCZoEdHHuF76DZ78IBJvlQ clear
namuwiki title : 신성조 clear
youtube title : 신성조 clear
channel description :  clear
keywords : IT 컴퓨터 PC 게임 신성조 조립컴 okay
category : ['Knowledge', 'Video_game_culture'] clear
published date : 2016-11-12T10:55:49Z clear
upload playlist id : UUNCZoEdHHuF76DZ78IBJvlQ clear
viewcount : 14628685 clear
subscriber count : 95000 clear
video count : 288 clear
no trailer video
maybe english
country : KR cle

In [13]:
# 채널 정보 저장하기(구버전)

import re

youtube = build('youtube','v3',developerKey=youtubeApiKey)

for i in range(len(channel_list['channel_ID'])):
    #db 구조
    channel_data = {
        'youtube_title' : '',
        'namuwiki_title' : '',
        'channel_id' : '',
        'channel_desc' : '',
        'category' : [],
        'keywords' : []
    }
    # 채널 id, 나무위키 제목 저장
    channel_data['channel_id'] = channel_list['channel_ID'][i]
    channel_data['namuwiki_title'] = channel_list['name_namuwiki'][i]
    print("channel id : {} . namuwiki : {}".format(channel_data['channel_id'], channel_data['namuwiki_title']))

    # YOUTUBE API 연결
    channels_api = youtube.channels().list(part='brandingSettings, topicDetails', id=channel_data['channel_id']).execute()
    print("api ok")

    # 유튜브 제목, 채널 소개 저장
    channel_data['youtube_title'] = channels_api['items'][0]['brandingSettings']['channel']['title']
    try:
        channel_data['channel_desc'] = channels_api['items'][0]['brandingSettings']['channel']['description']
    except:
        print('no description')
    print("youtube title : {}. \ndesc : {}".format(channel_data['youtube_title'], channel_data['channel_desc']))

    # 채널 키워드 저장
    try:
        keyword_list = channels_api['items'][0]['brandingSettings']['channel']['keywords'].split()
        for keyword in keyword_list:
            channel_data['keywords'].append(keyword)
        print("keywords {}".format(channel_data['keywords']))
    except :
        print("no keywords")

    # 카테고리 저장
    category_list =  channels_api['items'][0]['topicDetails']['topicCategories']
    for category in category_list:
        clean_category = re.sub(pattern='https://en.wikipedia.org/wiki/', repl='', string=category)
        channel_data['category'].append(clean_category)
    print('category : {}'.format(channel_data['category']))


    # firestore 저장
    # doc_title = re.sub(pattern=' ', repl='', string=channel_data['youtube_title'])
    doc_title = channel_data['youtube_title']
    youtuber_ref = db.collection(u'Youtuber').document(doc_title)
    youtuber_ref.set(channel_data)
    print(doc_title + ' firestore ok')




channel id : UChQ-VMvdGrYZxviQVMTJOHg . namuwiki : 도티
api ok
youtube title : 도티 TV. 
desc : *With SANDBOX Network*

▶인스타그램(@ddotty.heesun) : https://www.instagram.com/ddotty.heesun

채널을 구독 하시면 매일매일 업로드 되는 재밌는 영상들을 쉽고 편하게 확인 하실 수 있습니다!!
시청자분들과 함께 호흡하고 소통하는 채널이 될 수 있도록 성실히 노력하겠습니다 :D

#마인크래프트 #모바일게임 #종합게임 #Vlog

▷ 재생목록을 통해 영상을 시청하시면 시리즈 컨텐츠를 순서대로 편하게 보실 수 있습니다.

※각종 문의: ddotty@sandboxnetwork.net
keywords ['마인크래프트', '도티', '마크', '로블록스', '모바일게임', '클래시로얄']
category : ['Video_game_culture', 'Role-playing_video_game', 'Action_game']
도티 TV firestore ok
channel id : UCbFzvzDu17eDZ3RIeaLRswQ . namuwiki : 감스트
api ok
youtube title : 감스트GAMST. 
desc : 재미와 감동을 동시에 느낄 수 있는
감스트의 공식 유튜브 채널 (감튜브) 입니다!

감스트와 같이 찢으실 분들은
'구독' 눌러주시고 많이 시청하러 오세요 :-D

To have fun and be touched at the same time
The official YouTube channel of GAMST (GAMTUBE) *_*

If you want to have fun with GAMST
Please visit and subscribe to the channel for more videos
keywords ['감스트', 'gamst', '김인직', 'kiminjik', '먹방', 'mukbang', 'meokbang',

In [35]:
# 최신 동영상 저장하기(신버전)
#태그, 카테고리, 재생수, 좋아요수, 싫어요수, 댓글수는 video 메소드를 통해 다시 수집

import re

youtube = build('youtube','v3',developerKey=youtubeApiKey)

err_youtuber = []


#db에서 목록 가져오기
youtuber_ref = db.collection(u'Youtuber')
docs = youtuber_ref.stream()

for i, doc in enumerate(docs):
    if i < 113:
        continue
    
    video_data = {
        'video_title' : '',
        'video_id' : '',
        'video_desc' : '',
        'published_date' : '',
        'tags' : [],
        'video_category' : [],
        'view_count' : 0,
        'like_count' : 0,
        'dislike_count' : 0,
        'comment_count' : 0
    }

    doc_dic = doc.to_dict()
    doc_id = doc.id
    print('###############'+'current youtuber : ' + doc_id + 'number '+ str(i))


    #최근 업로드 playlist 가져오기
    recent_video_playlist_id = doc_dic['upload_playlist_id']
    
    #youtube api playlistitems 메소드 연결
    try:
        playlistitems = youtube.playlistItems().list(part='id, snippet, contentDetails, status', playlistId=recent_video_playlist_id,maxResults=50).execute()
    except:
        print('please check this youtuber###############')
        err_youtuber.append(doc_id)


    for video in playlistitems['items']:
        #video id 저장
        video_data['video_id'] = video['snippet']['resourceId']['videoId']
        # print('video id : ' + video_data['video_id'] + ' clear')

        #video title 저장
        video_data['video_title'] = video['snippet']['title']
        # print('video title : ' + video_data['video_title'] + ' clear')

        #video description 저장
        try:
            video_data['video_desc'] = video['snippet']['description']
            # print('video description : ' + ' clear')
        except:
            print('no description for video')

        #video 업로드일 저장
        video_data['published_date'] = video['snippet']['publishedAt']
        # print('video published date : '+ video_data['published_date']+ ' clear')


        #파이어스토어 저장
        
        video_ref = db.collection(u'Youtuber').document(doc_id).collection(u'Recent_videos')
        video_ref.add(video_data)
        # print('@@@@@@@@@@@@@' + 'this video store clear')
    
    print('all clear for this youtuber!!')
    print('err {}'.format(err_youtuber))


        




###############current youtuber : TheRankingsnumber 113
all clear for this youtuber!!
err []
old []
###############current youtuber : Today on the Korean Servernumber 114
all clear for this youtuber!!
err []
old []
###############current youtuber : Toon Tubenumber 115
all clear for this youtuber!!
err []
old []
###############current youtuber : ToyFamilynumber 116
all clear for this youtuber!!
err []
old []
###############current youtuber : Toycoo 토이쿠number 117
all clear for this youtuber!!
err []
old []
###############current youtuber : Wall Sunumber 118
all clear for this youtuber!!
err []
old []
###############current youtuber : Wolfnumber 119
all clear for this youtuber!!
err []
old []
###############current youtuber : Xboxnumber 120
all clear for this youtuber!!
err []
old []
###############current youtuber : YESOONnumber 121
all clear for this youtuber!!
err []
old []
###############current youtuber : YOU TUBE공갈number 122
all clear for this youtuber!!
err []
old []
##############

ResourceExhausted: 429 Quota exceeded.

In [3]:
# 인기 동영상 (최대)50개 저장하기(구버전)
# 영상 설명과 태그, 댓글은 다시 수집해야됨

import re

youtube = build('youtube','v3',developerKey=youtubeApiKey)

#db에서 목록 가져오기
youtuber_ref = db.collection(u'Youtuber')
docs = youtuber_ref.stream()

i = 0
max = 100

for doc in docs:
    if i < 20 :
        i = i+1
        continue
    elif i > max :
        break
    
    
    print("current number : {}".format(i))
    i = i+1

    doc_id = doc.id
    print("current youtuber : {}".format(doc_id))

    data = {
        'video_title' : '',
        'video_id' : '',
        'video_desc' : '',
        'tags' : []

    }
    doc_dic = doc.to_dict()
    channel_id = doc_dic['channel_id']

    search = youtube.search().list(part='id, snippet', channelId=channel_id, type='video', order='viewCount', maxResults='50').execute()
    print("YOUTUBE api ok")

    for video in search['items']:
        video_ref = db.collection(u'Youtuber').document(doc_id).collection(u'Popular_videos')

        data['video_id'] = video['id']['videoId']
        data['video_title'] = video['snippet']['title']
        try:
            data['video_desc'] = video['snippet']['description']
        except:
            print("no desc")
        print("data {}".format(data) )

        video_ref.add(data)
        print(channel_id + "firestore ok")
    
    print("youtuber {} OK===============".format(doc_id))







current number : 20
current youtuber : 글자네 YouTube
YOUTUBE api ok
data {'video_title': '시간끊기x 엔젤릭버스터 육성 Lv.200 45분컷 (테라버닝)', 'video_id': 'RGteRtBjOxA', 'video_desc': '구독, 좋아요 그리고 댓글 감사합니다! ------------------------------------------------------------------------------------------- 1. 분쟁유발, 비교, 특정인 비하 등등.. 댓글 차단합니다. 2.', 'tags': []}
UCb5NLtXAsTBrmaZVhyFa-Wgfirestore ok
data {'video_title': 'BJ세글자 샷건 레전드영상', 'video_id': 'QpVDqg6jh0o', 'video_desc': '', 'tags': []}
UCb5NLtXAsTBrmaZVhyFa-Wgfirestore ok
data {'video_title': '레벨과 스펙은 권력이 아닙니다.', 'video_id': 'uYadMeJcHIg', 'video_desc': '구독, 좋아요 그리고 댓글 감사합니다! ------------------------------------------------------------------------------------------- 1. 분쟁유발, 비교, 특정인 비하 등등.. 댓글 차단합니다. 2.', 'tags': []}
UCb5NLtXAsTBrmaZVhyFa-Wgfirestore ok
data {'video_title': '실검에 까지 올라간 장갑 먹튀사건... ★maplestory★', 'video_id': 'W6bo8JbkNCA', 'video_desc': '구독, 좋아요 그리고 댓글 감사합니다! 우리 스폰서 놀꽃 앱 다운로드 해주러 가기 : https://nolggot.com/market ...', 'tags': []}
UCb5NLtXAsTBrmaZ